<a href="https://colab.research.google.com/github/rhaagsma/ProjetoFinalPython/blob/insight-1/ProjetoFinal_Insight_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd

public_key = userdata.get('public_key')
private_key = userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest
    }

In [ ]:
base_url = "https://gateway.marvel.com/v1/public/characters"

response = requests.get(base_url, auth_params)
data = response.json()
df = pd.DataFrame(data['data']['results'])

In [ ]:
conn = sqlite3.connect('banco.db')
columns = ['id', 'name','description']

df[columns].to_sql('characters', conn, if_exists='replace', index=False)

conn.close()